In [22]:
%load_ext autoreload
%autoreload 2
import torch

import os
import sys
import IPython.display as ipd
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load models

In [3]:
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule

state_dict_path = '/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/optuna_90.5.p'
loaded_dict = torch.load(state_dict_path)
loaded_dict['hparams']['batch_size'] = 5
model = SpectrogramCNNPLModule(loaded_dict['hparams'])
model.model.load_state_dict(loaded_dict['state_dict'])
model = model.cuda()
model = model.eval()

# Load Data

In [5]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [6]:
datasetHandler.load(model, 'validation')

Loading cached validation data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_48k/


# Load Overlay data

In [10]:
import pickle

In [14]:
overlay_data_path = "/nfs/students/summer-term-2020/project-4/data/interpolation_data/data_48k/interpolation_data_downsampled.p"
overlay_data = pickle.load(open(overlay_data_path, "rb"))

In [18]:
classes = set([data['category'] for data in overlay_data])

In [19]:
overlay_dataset = {}
for data in overlay_data:
    overlay_dataset.setdefault(data['category'], []).append(data)

In [43]:
del overlay_dataset['Alarm']
classes.remove("Alarm")

# Attacks

In [47]:
fixed_epsilon = 0.002

## Functional Interpolation (perceptible)

In [ ]:
from attacks.functionalInterpolation import FunctionalInterpolationAttack
import torch.nn.functional as F

attack_results = {}

for overlay_class in classes:
    
    overlay_success_rate = 0 
    for overlay_sample in overlay_dataset[overlay_class]:
        overlay_sample = {'audio': torch.tensor(overlay_sample['data'][0])}
        audio = overlay_sample['audio']
        overlay_sample['audio'] = F.pad(audio, (0, 481489 - len(audio)), mode='constant', value=0)
        
        attack_parameters = {'overlay_sound': overlay_sample,
                         'epsilonInterpolation': 0.1, 'epsilonNoise': fixed_epsilon, 'num_iter': 1, 'upper2': 1,
                         }
        fva = FunctionalInterpolationAttack(model, model.val_dataloader(), attack_parameters, save_samples=False)
        fva.attack()
        overlay_success_rate += fva.getSuccessRate()
        
    overlay_success_rate /= len(overlay_dataset[overlay_class]) # /= 100
    attack_results[overlay_class] = overlay_success_rate

100%|██████████| 338/338 [00:16<00:00, 20.62it/s]


100%|██████████| 338/338 [00:16<00:00, 20.58it/s]

In [69]:
attack_results

{'Animal': 0.006292650918635171,
 'Motor vehicle (road)': 0.007053805774278216,
 'Music': 0.006148293963254593,
 'Natural sounds': 0.005354330708661418,
 'Human voice': 0.004435695538057743}

# Visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [ ]:
result = attack_results
result = {re.sub(r" ?\([^)]+\)", "", entry): [result[entry]*100] for entry in sorted(result, key = lambda x: result[x], reverse=True)}

In [ ]:
df = pd.DataFrame.from_dict(result)

sns.set()
sns.set_style("whitegrid")
sns.set_context("poster")

f = plt.figure(figsize=(15,10))
plt.ylabel("average success rate")
ax = sns.barplot(data=df)